In [1]:
import sys
sys.path.insert(0, '../')

from interface import prepare_tsv, save_index, top_n_similar, Collection, pair_scores, calculate_cosine_similarity

from typing import Tuple, List, Dict, Union, Any

from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.modeling.checkpoint import Checkpoint
from colbert.modeling.colbert import colbert_score
from colbert.data import Queries, Collection
from colbert import Indexer, Searcher

from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import torch
import os

from matplotlib import pyplot as plt


In [2]:
csv_models = '/home/sondors/Documents/price/BERT_data/data/models_17_categories.csv'
df_models = pd.read_csv(csv_models, sep=';')

csv_offers = '/home/sondors/Documents/price/offers_clusterization/data/14_categories_balanced.csv'
df_offers = pd.read_csv(csv_offers, sep=';')

id_category = {
    # 3902: 'диктофоны, портативные рекордеры',
    # 510402: 'электронные книги',
    # 4302: 'автомобильные телевизоры, мониторы',
    # 2815: 'смарт-часы и браслеты',
    # 3901: 'портативные медиаплееры',
    # 3904: 'портативная акустика',
    # 2801: 'мобильные телефоны',
    # 3908: 'VR-гарнитуры (VR-очки, шлемы, очки виртуальной реальности, FPV очки для квадрокоптеров)',
    # 510401: 'планшетные компьютеры и мини-планшеты',
    2102: 'наушники, гарнитуры, наушники c микрофоном',
    # 3903: 'радиоприемники, радиобудильники, радиочасы',
    # 3907: 'магнитолы',
    # 280801: 'GPS-навигаторы'
    }

df_models = df_models[df_models['category_id'].isin(id_category.keys())].reset_index(drop=True)
df_offers = df_offers[df_offers['category_id'].isin(id_category.keys())].reset_index(drop=True)

models = list(df_models['full_name'])
offers = list(df_offers['name'])

df_offers

,price,name,attrs,description,model_id,external_category,external_brand,external_type,category_name,category_id
0,8404.0,Игровая гарнитура Plantronics RIG 400HS Black,"[{'id': 9170, 'type': 'b', 'value': 1.0}]",100023001711,514599,Электроника :: Компьютеры и комплектующие :: А...,Plantronics,Игровая гарнитура,"наушники, гарнитуры, наушники c микрофоном",2102
1,4750.0,Компьютерная гарнитура Plantronics RIG 600 Dol...,[],"компьютерная гарнитура, игровая с полноразмерн...",514599,Электроника :: Портативная техника :: Наушники...,NaN,NaN,"наушники, гарнитуры, наушники c микрофоном",2102
2,11990.0,Наушники для Xbox One Plantronics RIG 800LX,"[{'id': 23564, 'type': 'b', 'value': 1.0}]","Тип подключения: беспроводной Радио, Частотный...",514599,XBOX :: Наушники для Xbox ONE,Plantronics,Наушники для Xbox One,"наушники, гарнитуры, наушники c микрофоном",2102
3,3990.0,Наушники для Xbox Plantronics RIG 400HX,"[{'id': 9163, 'type': 'b', 'value': 1.0}, {'id...","Тип подключения: проводной, Частотный диапазон...",514599,XBOX :: Наушники для Xbox ONE,Plantronics,Наушники для Xbox,"наушники, гарнитуры, наушники c микрофоном",2102
4,10416.0,"Гарнитура Plantronics RIG 500 PRO HC черная, 1...","[{'id': 9170, 'type': 'b', 'value': 1.0}]",NaN,514599,Телефония :: Телефония :: Plantronics решения ...,Plantronics,Гарнитура,"наушники, гарнитуры, наушники c микрофоном",2102
...,...,...,...,...,...,...,...,...,...,...
86485,5800.0,Наушники Monster Clarity 101 Airlinks MH21902 ...,"[{'id': 9163, 'type': 'b', 'value': 1.0}]",NaN,6200762,"Наушники и усилители, гарнитуры и аксессуары :...",Monster,Наушники,"наушники, гарнитуры, наушники c микрофоном",2102
86486,6990.0,Беспроводные наушники Monster Clarity 101 Airl...,[],NaN,6200762,Все товары :: Электроника :: Портативная техни...,Monster,Беспроводные наушники,"наушники, гарнитуры, наушники c микрофоном",2102
86487,3990.0,Наушники Monster Clarity 101 Airlinks MH21902 ...,"[{'id': 9170, 'type': 'b', 'value': 1.0}]",Основные характеристики Тип наушников: вакуумн...,6200762,Смартфоны и гаджеты :: Наушники,Monster,Наушники,"наушники, гарнитуры, наушники c микрофоном",2102
86488,7190.0,Беспроводные наушники Monster Clarity 101 Airl...,"[{'id': 9163, 'type': 'b', 'value': 1.0}]",Наушники Monster Bluetooth помогут вам остават...,6200762,Наушники :: Беспроводные наушники,Monster,Беспроводные наушники,"наушники, гарнитуры, наушники c микрофоном",2102


In [3]:
df_models

,model_id,name,brand_name,full_name,category_name,category_id
0,302182,BAZZ SL-8750,SpeedLink,SpeedLink BAZZ SL-8750,"наушники, гарнитуры, наушники c микрофоном",2102
1,302183,XBA-C10,Sony,Sony XBA-C10,"наушники, гарнитуры, наушники c микрофоном",2102
2,302207,H3080,Rapoo,Rapoo H3080,"наушники, гарнитуры, наушники c микрофоном",2102
3,302253,Call of Duty: Black Ops II Ear Force X-RAY,Turtle Beach,Turtle Beach Call of Duty: Black Ops II Ear Fo...,"наушники, гарнитуры, наушники c микрофоном",2102
4,312306,ATH-ESW11 LTD,Audio-Technica,Audio-Technica ATH-ESW11 LTD,"наушники, гарнитуры, наушники c микрофоном",2102
...,...,...,...,...,...,...
11113,6350836,TS-451M,Takstar,Takstar TS-451M,"наушники, гарнитуры, наушники c микрофоном",2102
11114,6350837,SC01,Tecno,Tecno SC01,"наушники, гарнитуры, наушники c микрофоном",2102
11115,6350838,TWS1UNC,Unico,Unico TWS1UNC,"наушники, гарнитуры, наушники c микрофоном",2102
11116,6350839,TWSBSUNC,Unico,Unico TWSBSUNC,"наушники, гарнитуры, наушники c микрофоном",2102


In [4]:
# ckpt_pth = "/home/sondors/Documents/ColBERT_weights/triples_X1_13_categories_use_ib_negatives/none/2024-01/26/10.49.44/checkpoints/colbert-5387-finish"
# experiment = "colbert-5387"

ckpt_pth = "/home/sondors/Documents/ColBERT_weights/bert-base-multilingual-cased_dim_768_bsize_230_lr04_use_ib_negatives/none/2024-01/27/16.55.29/checkpoints/colbert-2998-finish"
experiment = "colbert-2998-finish"

doc_maxlen = 300
nbits = 2   # bits определяет количество битов у каждого измерения в семантическом пространстве во время индексации
nranks = 1  # nranks определяет количество GPU для использования, если они доступны
kmeans_niters = 4 # kmeans_niters указывает количество итераций k-means кластеризации; 4 — хороший и быстрый вариант по умолчанию. 

device = "cuda"
config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits, kmeans_niters=kmeans_niters)
checkpoint = Checkpoint(ckpt_pth, colbert_config=config)
checkpoint.to(device)

Checkpoint(
  (model): HF_ColBERT(
    (linear): Linear(in_features=768, out_features=768, bias=False)
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(119547, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Lin

In [5]:
def get_query_emb(sentences: List[str], checkpoint: Checkpoint, batch_size: int) -> np.ndarray:
    """
    Учет контекста:
    - QueryTokenizer: Возможна передача контекста (например, фоновой информации) для запросов, и этот класс умеет обрабатывать такие случаи.
    - DocTokenizer: Обработка контекста обычно не требуется, поскольку документы сами по себе представляют отдельные единицы информации.
    """
    return checkpoint.queryFromText(sentences, bsize = batch_size).to("cpu").numpy()

def get_query_emb_batch(sentences: List[str], checkpoint: Checkpoint, batch_size: int, batch_size2: int) -> np.ndarray:
    # Создаем список для хранения эмбеддингов
    embeddings_list = []
    
    # Разбиваем список sentences на подсписки длины batch_size2
    for i in range(0, len(sentences), batch_size2):
        batch_sentences = sentences[i:i+batch_size2]
        print(f"batch: {i}")
        # Получаем эмбеддинги для текущего подсписка
        # embeddings = checkpoint.queryFromText(batch_sentences, bsize=batch_size).to("cpu").numpy()
        embeddings = get_query_emb(batch_sentences, checkpoint, batch_size)
        
        # Добавляем эмбеддинги в список
        embeddings_list.append(embeddings)
        torch.cuda.empty_cache()
    
    # Склеиваем полученные эмбеддинги по первой оси (по размерности batch_size2)
    combined_embeddings = np.concatenate(embeddings_list, axis=0)
    
    return combined_embeddings

offers_embs = get_query_emb_batch(offers, checkpoint, batch_size=100, batch_size2=10000)
# offers_embs = get_query_emb(offers, checkpoint, 3)


batch: 0
batch: 10000
batch: 20000
batch: 30000
batch: 40000
batch: 50000
batch: 60000
batch: 70000
batch: 80000


In [6]:
np.shape(offers_embs)

(86490, 32, 768)

In [7]:
models_embs = get_query_emb_batch(models, checkpoint, batch_size=100, batch_size2=10000)


batch: 0
batch: 10000


In [8]:
# models_embs = get_query_emb(models, checkpoint, 1)


In [9]:
offers_embs_mean = np.mean(offers_embs, axis=1)
models_embs_mean = np.mean(models_embs, axis=1)

In [10]:
np.shape(models_embs)

(11118, 32, 768)

In [11]:
np.shape(models_embs_mean)

(11118, 768)

In [12]:
torch.cuda.empty_cache()

In [13]:
sys.path.append("/home/sondors/Documents/price/ml-modelizer")
from modelizer.interface import top_n_similar, cosine_similarity_batch
import time
    
def plot_epochs_accuracy(filenames_int, accuracy_all, categories, y_axe='val_accuracy', x_axe='epoch'):
    plt.rcParams['figure.figsize'] = [15, 7]
    fig, ax = plt.subplots()

    for i in range(len(categories)):
        ax.plot(filenames_int, accuracy_all[i], label=categories[i])
    plt.xlabel(x_axe)
    plt.ylabel(y_axe)


    legend = ax.legend(loc='lower right', shadow=False, fontsize='x-small')

    # Put a nicer background color on the legend.
    legend.get_frame().set_facecolor('C0')
    plt.grid()
    plt.show()
    
def emb_to_df(df, embs):
    embed_name = []
    for index, row in df.iterrows():
        embed_name.append(embs[index])
    return embed_name

def calc_acc_for_category(df_offers, df_models, imya_kategorii, models_embs, offers_embs):
    print(f"{imya_kategorii}")
    start_time = time.time()
    df_offers = df_offers[df_offers.category_name == imya_kategorii]
    df_models = df_models[df_models.category_name == imya_kategorii]

    offer_embs = emb_to_df(df_offers, offers_embs)
    offer_model_id = list(df_offers['model_id'])

    model_embs = emb_to_df(df_models, models_embs)
    model_model_id = list(df_models['model_id'])

    print(f"\tофферов {len(offer_embs)} шт поиск матча для каждого среди {len(model_model_id)} шт моделей")

    batch_size = max((1, (5000*4000)//len(model_model_id)))
    top_n = top_n_similar(offer_embs, model_embs, model_model_id, batch_size, n=5)
    offer_model_id_predicted = [i['model_ids'][0] for i in top_n]

    equal = np.sum(np.array(offer_model_id) == np.array(offer_model_id_predicted))
    accuracy = round(100*equal/len(offer_model_id), 2)
    print(f"\taccuracy = {accuracy}%\n\ttime_spent = {round(time.time() - start_time, 2)} sec\n")
    return accuracy


accuracy_one_model = []
for category in id_category.values():
    accuracy = calc_acc_for_category(df_offers, df_models, category, models_embs_mean, offers_embs_mean)
    accuracy_one_model.append(accuracy)


наушники, гарнитуры, наушники c микрофоном
	офферов 86490 шт поиск матча для каждого среди 11118 шт моделей
	accuracy = 94.74%
	time_spent = 58.63 sec

